In [2]:
import requests
import chess.pgn
import io
import json

# 🎯 Define your target and parameters
username = "drnykterstein"
api_url = f"https://lichess.org/api/games/user/{username}"

params = {
    "max": 50,
    "rated": "true",
    "perfType": "blitz",
    "pgnInJson": "false"
}

# 📡 Make the GET request to the Lichess API
print("Fetching game data from Lichess...")
response = requests.get(api_url, params=params)

all_games_data = []

if response.status_code == 200:
    print("Successfully fetched data. Now parsing games...")
    pgn_text = response.text
    pgn_file = io.StringIO(pgn_text)

    # Loop through each game in the PGN text
    while True:
        game = chess.pgn.read_game(pgn_file)
        if game is None:
            break

        # 📊 Extract metadata features
        game_features = {
            "Result": game.headers.get("Result"),
            "WhiteElo": int(game.headers.get("WhiteElo", 0)),
            "BlackElo": int(game.headers.get("BlackElo", 0)),
            "TimeControl": game.headers.get("TimeControl"),
            "Termination": game.headers.get("Termination"),
            "Opening": game.headers.get("Opening"),
        }

        # ♟️ NEW: Extract movement data
        san_moves = []
        board = game.board() # Create a board from the game's starting position
        for move in game.mainline_moves():
            san_moves.append(board.san(move))
            board.push(move) # Update the board with the move

        game_features["Moves"] = san_moves
        game_features["TotalMoves"] = board.fullmove_number
        
        all_games_data.append(game_features)
    
    print(f"Successfully parsed {len(all_games_data)} games.")

    # 📝 Save the structured data to a JSON file
    output_filename = f"{username}_games_with_moves.json"
    with open(output_filename, 'w') as json_file:
        json.dump(all_games_data, json_file, indent=4)
    
    print(f"Data saved to {output_filename}")

else:
    print(f"Error: Failed to fetch data. Status code: {response.status_code}")
    print(response.text)


Fetching game data from Lichess...
Successfully fetched data. Now parsing games...
Successfully parsed 50 games.
Data saved to drnykterstein_games_with_moves.json
